# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [13]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [14]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [15]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'local', 'petsitly_marketing', 'uk_food']


In [16]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [17]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [18]:
# review a document in the establishments collection
print(db.establishments.find_one())

{'_id': ObjectId('64d82e799a15b3b69c2c22cf'), 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': '', 'AddressLine1': 'Penang Flavours', 'AddressLine2': '146A Plumstead Rd', 'AddressLine3': 'London', 'AddressLine4': '', 'PostCode': 'SE18 7DY', 'Phone': '', 'LocalAuthorityCode': '511', 'LocalAuthorityName': 'Greenwich', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'}, 'RightToReply': '', 'Distance': 4623.972328074718, 'NewRatingPending': True}


In [19]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [20]:
# Create a dictionary for the new restaurant data
pf ={
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [21]:
# Insert the new restaurant into the collection
establishments.insert_one(pf)

In [22]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({"BusinessName" : "Penang Flavours"}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64d82e799a15b3b69c2c22cf'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [24]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType" : "Restaurant/Cafe/Canteen"}
fields = {'BusinessTypeID' : 1 , 'BusinessType' : 1}

results = establishments.find(query,fields)

for i in range(5):
    pprint(results[i])

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 '_id': ObjectId('64d82e799a15b3b69c2c22cf')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 '_id': ObjectId('64d82eb79a15b3b69c2c22d1')}


IndexError: no such item for Cursor instance

3. Update the new restaurant with the `BusinessTypeID` you found.

In [25]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {'BusinessName': 'Penang Flavours'},
    {'$set': 
        {'BusinessTypeID': 1}}
)

In [26]:
# Confirm that the new restaurant was updated
query = {"BusinessName" : "Penang Flavours"}

fields = {"BusinessName" : "Penang Flavours" , 'BusinessTypeID' : 1 , 'BusinessType' : 1 }

establishments.find_one(query , fields)

{'_id': ObjectId('64d82e799a15b3b69c2c22cf'),
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'BusinessName': 'Penang Flavours'}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [27]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName' : 'Dover'}

results = establishments.find(query)

for i in range(5):
    pprint(results[i])

IndexError: no such item for Cursor instance

In [28]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(
    {'LocalAuthorityName': 'Dover'}
)

In [29]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName' : {'$regex' : 'Dover'}}

pprint(establishments.find_one(query))

None


In [30]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one({}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64d82e799a15b3b69c2c22cf'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [31]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({} , [{'$set': {'geocode.longitude' : {'$toDecimal' : '$geocode.longitude'}}}])

Use `update_many` to convert `RatingValue` to integer numbers.

In [32]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [33]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({} , [{'$set': {"geocode.latitude" : {'$toDecimal' : "$geocode.latitude"}}}])

In [34]:
# Check that the coordinates and rating value are now numbers
fields = {'geocode'}
pprint(establishments.find_one({} , fields))

{'_id': ObjectId('64d82e799a15b3b69c2c22cf'),
 'geocode': {'latitude': Decimal128('51.49014200'),
             'longitude': Decimal128('0.08384000')}}
